# Project 3 Group 5 

Ting Cai, Yicheng Li, Wenyue Wu, Kangkang Zhang, Na Zhuo

***
Import all relevant libraries:

In [1]:
import os
import copy
import pandas as pd
import sys
import numpy as np
import pickle
import matplotlib.pyplot as plt
import lightgbm as lgb
import time
import multiprocessing
from scipy.io import loadmat
from scipy.spatial.distance import pdist, squareform
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.ensemble import GradientBoostingClassifier  #GBM algorithm
from sklearn.decomposition import PCA
from IPython.core.interactiveshell import InteractiveShell
from scipy.io import loadmat
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Flatten, Conv1D, MaxPooling1D, Dropout, Reshape
from tensorflow.keras.regularizers import l1,l2


#import custom defined functions
sys.path.append("..")

import lib.feature as ft

/opt/anaconda3/lib/python3.7/site-packages/lightgbm/__init__.py:48: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


***
## Initialize relevant files and parameters

In [2]:
#interactive setting
InteractiveShell.ast_node_interactivity = "all"

#training data labels
info = pd.read_csv('../data/train_set/label.csv',usecols = range(1,6))

#parameters
rand_seed = 123
test_size = 0.2
cv_folds = 5
num_cores = multiprocessing.cpu_count()
all_idx = np.array(info.index)

Read in all the points from matrices and construct data frame

In [3]:
def readMat(index):
    thisMat = loadmat('../data/train_set/points/' + '%04d' % index + '.mat')
    return pd.DataFrame(round(pd.DataFrame(thisMat[list(thisMat)[3]]),0))

fiducial_pt_list = list(map(readMat, list(range(1,2501))))
#save and load file
f = open('../output/fiducial_pt_list', 'wb')
pickle.dump(fiducial_pt_list, f)
f.close()
f = open('../output/fiducial_pt_list', 'rb')
fiducial_pt_list = pickle.load(f)
f.close()

dat_full = ft.feature(copy.deepcopy(fiducial_pt_list),all_idx,info)

In [4]:
dat_full.head()

,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,feature10,...,feature5998,feature5999,feature6000,feature6001,feature6002,feature6003,feature6004,feature6005,feature6006,emotion_idx
0,45.0,28.0,6.0,16.0,37.0,22.0,1.0,25.0,176.0,136.0,...,111.0,168.0,225.0,56.0,113.0,170.0,57.0,114.0,57.0,1
1,44.0,27.0,5.0,19.0,36.0,20.0,2.0,24.0,170.0,134.0,...,111.0,169.0,228.0,58.0,116.0,175.0,58.0,117.0,59.0,1
2,42.0,24.0,3.0,18.0,33.0,15.0,5.0,23.0,153.0,118.0,...,102.0,156.0,211.0,53.0,107.0,162.0,54.0,109.0,55.0,1
3,47.0,27.0,7.0,15.0,31.0,13.0,8.0,29.0,166.0,126.0,...,112.0,173.0,234.0,60.0,121.0,182.0,61.0,122.0,61.0,1
4,35.0,22.0,1.0,19.0,36.0,19.0,0.0,18.0,161.0,122.0,...,102.0,157.0,211.0,52.0,107.0,161.0,55.0,109.0,54.0,1


Splitting into X/y, training/test

In [5]:
X, y = dat_full.iloc[:,:-1],dat_full.iloc[:,-1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=rand_seed)

***
## Model Training and Testing

****All models are saved and loaded to reduce waiting time during presentation. Uncomment the code in appropriate sections and rerun if needed.***

### Baseline Model: Boosted Decision Stumps 

Implemented using GradientBoostingClassifier() from sklearn.ensemble 

In [6]:
#baseGBM = GradientBoostingClassifier(random_state=rand_seed,max_depth = 1)
#startTime = time.time()
#baseGBM.fit(X_train, y_train)
#endTime= time.time()-startTime
#baseGBM_prediction = baseGBM.predict(X_test)
#baseGBM_accuracy = accuracy_score(y_test,baseGBM_prediction)
#
##save/load output
#baseGBMOut = {'Model':baseGBM, 'accuracy':baseGBM_accuracy,'Time':endTime}
#f = open('../output/baseGBMOut', 'wb')
#pickle.dump(baseGBMOut, f)
#f.close()

f = open('../output/baseGBMOut', 'rb')
baseGBMOut = pickle.load(f)
f.close()

print("Base GBM Fit Time is: " + str(int(baseGBMOut['Time']//60)) + " minutes and " + str(round(baseGBMOut['Time'] % 60)) + " seconds. Accuracy is: " + 
      str(baseGBMOut['accuracy']*100) + "%")

Base GBM Fit Time is: 2 minutes and 20 seconds. Accuracy is: 37.8%


Fit using entire dataset:

In [7]:
# baseGBM_final = GradientBoostingClassifier(random_state=rand_seed,max_depth = 1)
# baseGBM_final.fit(X, y)
# f = open('../output/baseGBM_final', 'wb')
# pickle.dump(baseGBM_final, f)
# f.close()
f = open('../output/baseGBM_final', 'rb')
baseGBM_final = pickle.load(f)
f.close()

****The paramter tuning process was took extremely long time and did not finish on time, so we only included the code. The tuned parameters in the following section were concluded based on tuning of smaller scales, trial-error, and intuitions.***

### Model Selection

In [8]:
data = {'Model':['GBM','SVM (linear)', 'SVM (poly)', 'SVM (rbf)', 'Xgboost', 'Neural Net', 'CNN','Random Forest'], 
        'Accuracy':['37.8%', '50.2%', '48.0%', '47.6%', '51.0%', '51.2%', '24.0%','41.0%'],
        'Running Time':['2m2s', '40s', '40s','2s', '1m5s', '19s', '21s', '17s']} 

model_Select= pd.DataFrame(data)
model_Select

,Model,Accuracy,Running Time
0,GBM,37.8%,2m2s
1,SVM (linear),50.2%,40s
2,SVM (poly),48.0%,40s
3,SVM (rbf),47.6%,2s
4,Xgboost,51.0%,1m5s
5,Neural Net,51.2%,19s
6,CNN,24.0%,21s
7,Random Forest,41.0%,17s


***

### Improved Model : Neural Net



#### Introduce New Feature

Based on the original feature: 

vertical distance and horizontal distance between any two points among those 78 fiducial points, 

We introduced new feature: 

the angle between any two-point vectors and x-axis. 

Therefore, we have 6006 + 3003 = 9009 features now.

Import matlab matrixs and calculate new features

In [9]:
# path for mats and label.csv
path = '../data/train_set/'
train_pt_dir = path + "points/"

In [10]:
# import matlab matrix and round to int
index = ['{0:04}'.format(num) for num in range(1, 2501)]

mats = []
for ind in index:
    temp = loadmat( train_pt_dir + ind + ".mat")
    mats.append(temp[[*temp][-1]])
    
mats = [mat.round() for mat in mats]

In [11]:
feature_dist_df = pd.DataFrame(ft.feature_distance(mats))

In [12]:
feature_slope_df = pd.DataFrame(ft.feature_slope(mats))

In [13]:
feature_all_df = pd.concat([feature_dist_df, feature_slope_df], axis=1, ignore_index=True)

Here we have a feature dataframe with 9009 columns.

**Apply PCA to reduce feature dimension**

We split the train set into 80%:20%, apply PCA to 80% train set to train the neural net model

In [14]:
X, y = feature_all_df, dat_full["emotion_idx"]
y = np.asarray(y - 1)

X_train, X_test, y_train, y_test = train_test_split(X, np.asarray(y), test_size=0.2, random_state=123)

pca = PCA(n_components=0.99, whiten=True)
X_pca_train = pca.fit_transform(X_train)
X_pca_test = pca.transform(X_test)
print(len(X_pca_train[1]))
pc_train_dim = len(X_pca_train[1])

121


Find best tuning parameter

In [15]:
model2 = Sequential()
model2.add(Dense(1000, input_dim=pc_train_dim, activation='tanh',kernel_initializer = 'lecun_uniform',activity_regularizer=l1(0.001)))
model2.add(Dense(600, activation='tanh',kernel_initializer = 'lecun_uniform'))
model2.add(Dense(600, activation='tanh',kernel_initializer = 'lecun_uniform'))
model2.add(Dense(600, activation='tanh',kernel_initializer = 'lecun_uniform'))
model2.add(Dense(600, activation='tanh',kernel_initializer = 'lecun_uniform'))
model2.add(Dense(400, activation='tanh',kernel_initializer = 'lecun_uniform'))
model2.add(Dropout(0.2))
model2.add(Dense(22, activation = 'sigmoid'))

model2.summary()

model2.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model2.fit(X_pca_train, y_train, epochs = 15)

pred2 = model2.predict(X_pca_test)
pred_index2 = np.argmax(pred2, axis = 1)
accuracy2 = accuracy_score(y_test, pred_index2)
accuracy2

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1000)              122000    
_________________________________________________________________
dense_1 (Dense)              (None, 600)               600600    
_________________________________________________________________
dense_2 (Dense)              (None, 600)               360600    
_________________________________________________________________
dense_3 (Dense)              (None, 600)               360600    
_________________________________________________________________
dense_4 (Dense)              (None, 600)               360600    
_________________________________________________________________
dense_5 (Dense)              (None, 400)               240400    
_________________________________________________________________
dropout (Dropout)            (None, 400)               0

0.512

Now fit using the entire dataset: feature and the optimal tuning parameter; 

save the model parameter for later use.

In [16]:
X, y = feature_all_df, dat_full["emotion_idx"]
#X, y = feature_dist_df, label_df["emotion_idx"]
y = np.asarray(y - 1)

from sklearn.decomposition import PCA
pca = PCA(n_components=0.99, whiten=True)
X_pca = pca.fit_transform(X)
print(len(X_pca[1]))
pc_dim = len(X_pca[1])

123


In [17]:
model_main = Sequential()
model_main.add(Dense(1000, input_dim=pc_dim, activation='tanh',kernel_initializer = 'lecun_uniform',activity_regularizer=l1(0.001)))
model_main.add(Dense(600, activation='tanh',kernel_initializer = 'lecun_uniform'))
model_main.add(Dense(600, activation='tanh',kernel_initializer = 'lecun_uniform'))
model_main.add(Dense(600, activation='tanh',kernel_initializer = 'lecun_uniform'))
model_main.add(Dense(600, activation='tanh',kernel_initializer = 'lecun_uniform'))
model_main.add(Dense(400, activation='tanh',kernel_initializer = 'lecun_uniform'))
model_main.add(Dropout(0.2))
model_main.add(Dense(22, activation = 'sigmoid'))

model_main.summary()

model_main.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model_main.fit(X_pca, y, epochs = 15)

model_main.save('../output/neural_network.h5')

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 1000)              124000    
_________________________________________________________________
dense_8 (Dense)              (None, 600)               600600    
_________________________________________________________________
dense_9 (Dense)              (None, 600)               360600    
_________________________________________________________________
dense_10 (Dense)             (None, 600)               360600    
_________________________________________________________________
dense_11 (Dense)             (None, 600)               360600    
_________________________________________________________________
dense_12 (Dense)             (None, 400)               240400    
_________________________________________________________________
dropout_1 (Dropout)          (None, 400)              

In [18]:
loaded = load_model(path + 'neural_network.h5')
df = pd.read_csv('../data/test_set_sec1/labels_prediction.csv')

Apply the model to the test set

In [19]:
test_pt_dir = "../data/test_set_sec1/points/"
# import matlab matrix and round to int
ntest_mat = 2500
index = ['{0:04}'.format(num) for num in range(1, ntest_mat + 1)]

test_mats = []

In [20]:
for ind in index:
    temp = loadmat( test_pt_dir + ind + ".mat")
    test_mats.append(temp[[*temp][-1]])
    
test_mats = [mat.round() for mat in test_mats]

In [21]:
t_feature_dist_df = pd.DataFrame(ft.feature_distance(test_mats))
t_feature_slope_df = pd.DataFrame(ft.feature_slope(test_mats))
t_feature_all_df = pd.concat([t_feature_dist_df, t_feature_slope_df], axis=1, ignore_index=True)

baseline model prediction

In [22]:
base_pred = baseGBM_final.predict(t_feature_dist_df)
df['Baseline'] = base_pred

advanced model prediction

In [23]:
tX = t_feature_all_df

#tpca = PCA(n_components=0.99, whiten=True)
tX_pca = pca.transform(tX)
print(len(tX_pca[1]))
tpc_dim = len(tX_pca[1])

#tpred = loaded.predict(tX)
#tpred_index = np.argmax(tpred, axis = 1) + 1

123


In [24]:
tpred = loaded.predict(tX_pca)
tpred_index = np.argmax(tpred, axis = 1) + 1

In [25]:
df['Advanced'] = tpred_index

In [27]:
df.to_csv('../data/test_set_sec1/labels_prediction.csv')